In [1]:
# CELL 1: Full corrected code, now importing the missing functions

from ipyleaflet import Map, basemaps, Marker, GeoJSON, Popup
from IPython.display import display, Markdown
import requests
import json

# PyNHD import for flowlines and streamgages
from pynhd import NLDI

# Import streamgage popup module
from gage_popup_utils import create_enhanced_popup_content

# Import StreamCat reporting
from streamcat_data import display_streamcat_report_for_comid

# Import the functions that were previously in cell 3
# (adjust the file name and import statement to match your setup)
from watershed_utils import format_watershed_report, display_map_layers

# 1. Create Interactive Map
center_usa = (47.897, -123.111)  # Approximate center of USA
zoom_level = 9

usa_map = Map(center=center_usa, zoom=zoom_level, basemap=basemaps.Esri.WorldStreetMap)
display(usa_map)

def handle_map_click(event, **kwargs):
    """Handles map click, retrieves COMID from NLDI,
    fetches Watershed Report, and displays formatted report and map layers.
    Afterward, it also displays the StreamCat report."""
    if kwargs.get('type') == 'click':
        latlng = kwargs.get('coordinates')
        latitude = latlng[0]
        longitude = latlng[1]

        # NLDI API URL for COMID
        nldi_url = f"https://api.water.usgs.gov/nldi/linked-data/comid/position?coords=POINT({longitude} {latitude})"
        api_key = 'grRcWoOIWpPgTCG40v4MFRxR1MtxPfNOrkltHvY8'  # Your API key

        try:
            response_nldi = requests.get(url=nldi_url, headers={'accept': 'application/json'})
            response_nldi.raise_for_status()
            nldi_data = response_nldi.json()
            comid = None
            features = nldi_data.get('features', [])
            if features:
                properties = features[0].get('properties', {})
                comid = properties.get('comid')

            if comid:
                print(f"Clicked at Latitude: {latitude}, Longitude: {longitude}")
                print(f"NLDI COMID: {comid}")

                # --- EPA WATERS API Request (Watershed Report v2.5) ---
                waters_api_url = f"https://api.epa.gov/waters/v2_5/nhdplus_json?pcomid={comid}&f=json"
                headers = {'X-Api-Key': api_key}
                try:
                    # Debug prints
                    print(f"[DEBUG] Requesting WATERS at {waters_api_url} ...")
                    response_waters = requests.get(waters_api_url, headers=headers)
                    print(f"[DEBUG] WATERS response status code: {response_waters.status_code}")
                    response_waters.raise_for_status()

                    waters_data = response_waters.json()
                    # Debug info
                    print(f"[DEBUG] WATERS response keys: {list(waters_data.keys())}")

                    # Format and Display the Watershed Report
                    formatted_report = format_watershed_report(waters_data)
                    display(formatted_report)

                    # --- Fetch and display map layers ---
                    display_map_layers(comid, usa_map)

                    # --- Display the StreamCat report for this COMID ---
                    display_streamcat_report_for_comid(comid)

                except requests.exceptions.RequestException as e_waters:
                    print(f"Error: Failed to connect to EPA WATERS API (v2.5). {e_waters}")
                except json.JSONDecodeError:
                    print("Error: Could not decode JSON response from EPA WATERS API (v2.5).")

            else:
                print("Error: COMID not found in NLDI response. Cannot proceed to WATERS API.")

        except requests.exceptions.RequestException as e_nldi:
            print(f"Error: Failed to connect to NLDI API. {e_nldi}")
        except json.JSONDecodeError:
            print("Error: Could not decode JSON response from NLDI API.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

usa_map.on_interaction(handle_map_click)

print("Click on the map to get NLDI COMID, Watershed data (v2.5), Streamgage popup info, and StreamCat report!")

Map(center=[47.897, -123.111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Click on the map to get NLDI COMID, Watershed data (v2.5), Streamgage popup info, and StreamCat report!


Clicked at Latitude: 47.964180715412276, Longitude: -123.10455322265626
NLDI COMID: 23997454
[DEBUG] Requesting WATERS at https://api.epa.gov/waters/v2_5/nhdplus_json?pcomid=23997454&f=json ...
[DEBUG] WATERS response status code: 200
[DEBUG] WATERS response keys: ['output', 'status']


### For the stream segment
| Attribute | Value | Unit |
|---|---|---|
| Stream Name | Dungeness River | None |
| Stream Order | 3 | None |
| Stream Level | 1 | None |
| Mean annual flow volume (estimate) | 289.03 |  cfs |
| Mean annual flow velocity (estimate) | 1.80267 |  fps |
| Stream Length | 2.26972267367192 |  km |
| Stream Time of Travel (estimate) | 0.04781685 |  days |

#### Category: For the catchment (local area draining directly to the selected stream segment)
##### Subcategory: Metrics
| Attribute | Value | Unit |
|---|---|---|
| Catchment area measurement | 2.53271236082945 |  km² |
| Mean annual temperature | 9.23841979 |  °C |
| Mean annual precipitation | 1086.26 |  mm |

##### Subcategory: 2011 National Land Cover Dataset
| Attribute | Value | Unit |
|---|---|---|
| Open Water (11) | 0 | % |
| Low Intensity Residential (21) | 2.313991 | % |
| Commercial (23) | 0 | % |
| Deciduous Forest (41) | 0 | % |
| Evergreen Forest (42) | 84.834461 | % |
| Mixed Forest (43) | 2.313991 | % |
| Other | 10.537558 | % |

#### Category: For the watershed (drainage area extending from the outlet upstream to the headwaters)
##### Subcategory: Metrics
| Attribute | Value | Unit |
|---|---|---|
| Drainage area measurement | 185.1129 |  km² |
| Mean annual temperature | 5.31611943 |  °C |
| Mean annual precipitation | 1955.50344514 |  mm |

##### Subcategory: 2011 National Land Cover Dataset
| Attribute | Value | Unit |
|---|---|---|
| Open Water (11) | 0.025282 | % |
| Low Intensity Residential (21) | 0.392841 | % |
| Commercial (23) | 0 | % |
| Deciduous Forest (41) | 0.087028 | % |
| Evergreen Forest (42) | 71.27688 | % |
| Mixed Forest (43) | 0.315537 | % |
| Other | 27.90243 | % |



### StreamCat Data Report

#### Landscape Layer: 2006 National Land Cover Database Agricultural Land Cover on Slopes
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent agricultural land cover (classes 81 and 82) of the catchment area classified by the NLCD2006 landscape raster where slope is greater than 10%. | 0% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the watershed area classified by the NLCD2006 landscape raster where slope is greater than 10%. | .0165304524968276% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the catchment area classified by the NLCD2006 landscape raster where slope is equal to or greater than 20%. | 0% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the watershed area classified by the NLCD2006 landscape raster where slope is equal to or greater than 20%. | .0165304524968276% | 100 |

#### Landscape Layer: Agricultural Nitrogen Inputs
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean rate of biological nitrogen fixation from the cultivation of crops in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of biological nitrogen fixation from the cultivation of crops in kg N/ha/yr within the total upstream watershed. | .00247956787452414kg/ha/yr | 100 |
| Mean rate of synthetic nitrogen fertilizer application in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of synthetic nitrogen fertilizer application in kg N/ha/yr within the total upstream watershed. | .00512444027401656kg/ha/yr | 100 |
| Mean rate of manure application from confined animal feeding operations in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of manure application from confined animal feeding operations in kg N/ha/yr within the total upstream watershed. | .000661218099873104kg/ha/yr | 100 |

#### Landscape Layer: Base Flow Index
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Ratio of base flow to total flow, expressed as a percentage within the local catchment. | 59.9316482734069% | 100 |
| Ratio of base flow to total flow, expressed as a percentage within the total upstream watershed. | 58.6769609249274% | 100 |

#### Landscape Layer: Canal Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of NHDPlusV2 line features within the local catchment classified as canal, ditch, or pipeline.(kilometer of canal/square kilometer). | 0km/km² | 100 |
| Density of NHDPlusV2 line features within the total upstream watershed classified as canal, ditch, or pipeline.(kilometer of canal/square kilometer). | 0km/km² | 100 |

#### Landscape Layer: National Coal Resource Dataset System
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced coal mine sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced coal mine sites within the total upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: Dam Density and Storage Volume
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced dams within the local catchment. | 0dams/km² | 100 |
| Density of georeferenced dams within the upstream watershed. | 0dams/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the local catchment. | 0m³/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the total upstream watershed. | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the local catchment. | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the total upstream watershed. | 0m³/km² | 100 |

#### Landscape Layer: National Elevation Dataset
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all elev_cm values within the local catchment divided by 100 to convert cm to m. | 494.38259167m | 100 |
| Mean of all elev_cm values within the upstream watershed divided by 100 to convert cm to m. | 1285.83588056m | 100 |

#### Landscape Layer: Facility Registry Services (FRS): Toxic Release Inventory (TRI), National Pollutant Discharge Elimination System (NPDES), and Superfund Sites
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced National Pollutant Discharge Elimination System sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: Wildland Fire Perimeters By Year 2000 - 2010
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage forest loss to fire (fire perimeter) in 2000 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the riparian mask of the total upstream watershed. | 0% | 100 |

#### Landscape Layer: Forest Loss By Year 2001 to 2013
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent of forest loss detected primarily in the year 2001 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2001 within the total upstream watershed. | .00145856933796% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the total upstream watershed. | .00291713867591% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the total upstream watershed. | .00875141602773% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the total upstream watershed. | .00194475911727% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the total upstream watershed. | .00145856933796% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the total upstream watershed. | .00437570801387% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the total upstream watershed. | .0491051677112% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the total upstream watershed. | .00243094889659% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the total upstream watershed. | .0053480875725% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the total upstream watershed. | .0150718831589% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the total upstream watershed. | .0160442627175% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the total upstream watershed. | .000972379558637% | 100 |

#### Landscape Layer: Riparian Buffer Forest Loss By Year 2001 to 2013
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent of forest loss detected primarily in the year 2001 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2001 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the riparian mask of the total upstream watershed. | .00406586704614759% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the riparian mask of the total upstream watershed. | .069119739784509% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the riparian mask of the total upstream watershed. | .0284610693230331% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the riparian mask of the total upstream watershed. | .00813173409229518% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the riparian mask of the total upstream watershed. | .0325269363691807% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the riparian mask of the total upstream watershed. | 0% | 100 |

#### Landscape Layer: Olson and Hawkins (2014) Geochemical and Geophysical Characteristics
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Lithological aluminum oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 10.4149904250979% | 100 |
| Lithological aluminum oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 11.7793917479981% | 100 |
| Lithological calcium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 1.6239409446707% | 100 |
| Lithological calcium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 5.78188452438485% | 100 |
| Lithological ferric oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 3.78183889387682% | 100 |
| Lithological ferric oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 6.42253353518312% | 100 |
| Lithological potassium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 1.99170196056248% | 100 |
| Lithological potassium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 1.45912103899242% | 100 |
| Lithological magnesium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 1.69679403305091% | 100 |
| Lithological magnesium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 3.65376460009432% | 100 |
| Lithological sodium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | .820595026016376% | 100 |
| Lithological sodium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 1.69860265547134% | 100 |
| Lithological phosphorus pentoxide content in surface or near surface geology, expressed as a percentage within the local catchment. | .143553003668921% | 100 |
| Lithological phosphorus pentoxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .210725154466382% | 100 |
| Lithological sulfur content in surface or near surface geology, expressed as a percentage within the local catchment. | .116467997431826% | 100 |
| Lithological sulfur content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .0390367712838813% | 100 |
| Lithological silicon dioxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 67.8745651245995% | 100 |
| Lithological silicon dioxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 61.3325765855864% | 100 |
| Lithological nitrogen content in surface or near surface geology, expressed as a percentage within the local catchment. | .0762709975243859% | 100 |
| Lithological nitrogen content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .0241130172657173% | 100 |
| Hydraulic conductivity (in micrometers per second) of surface or near surface geology, within the local catchment. | .212121993303311% | 100 |
| Hydraulic conductivity (in micrometers per second) of surface or near surface geology, within the total upstream watershed. | .412261695476004% | 100 |
| Compressive strength, measured as uniaxial compressive strength (in megaPascals) of surface or near surface geology, within the local catchment | 78.2626419067284% | 100 |
| Compressive strength, measured as uniaxial compressive strength (in megaPascals) of surface or near surface geology, within the total upstream watershed | 135.276939820401% | 100 |

#### Landscape Layer: 2006 National Land Cover Database Impervious Surfaces
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean imperviousness of anthropogenic surfaces within catchment. | .241011035955856% | 100 |
| Mean imperviousness of anthropogenic surfaces within watershed. | .0421186205823581% | 100 |
| Mean imperviousness of anthropogenic surfaces within catchment and within a 100-m buffer of NHD stream lines. | .135755258126195% | 100 |
| Mean imperviousness of anthropogenic surfaces within watershed and within a 100-m buffer of NHD stream lines. | .0425696279731653% | 100 |

#### Landscape Layer: Soil Erodibility (KFFACT)
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean agricultural surface soil erodibility factor adjusted within the local catchment. | 0 | 100 |
| Mean agricultural surface soil erodibility factor adjusted within the total upstream watershed. | .0000446322217414345 | 100 |
| Mean surface soil erodibility factor adjusted within the local catchment. | .27 | 100 |
| Mean surface soil erodibility factor adjusted within the total upstream watershed. | .218153159504281 | 100 |

#### Landscape Layer: Surficial Lithology
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of catchment area classified as lithology type: alkaline intrusive volcanic rock. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: alkaline intrusive volcanic rock. | 0% | 100 |
| Percentage of catchment area classified as lithology type: alluvium and fine-textured coastal zone sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: alluvium and fine-textured coastal zone sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: carbonate residual material. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: carbonate residual material. | 0% | 100 |
| Percentage of catchment area classified as lithology type: coastal zone sediment, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: coastal zone sediment, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: colluvial sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: colluvial sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: eolian sediment, coarse-textured (sand dunes). | 0% | 100 |
| Percentage of watershed area classified as as lithology type: eolian sediment, coarse-textured (sand dunes). | 0% | 100 |
| Percentage of catchment area classified as lithology type: eolian sediment, fine-textured (glacial loess). | 0% | 100 |
| Percentage of watershed area classified as as lithology type: eolian sediment, fine-textured (glacial loess). | 0% | 100 |
| Percentage of catchment area classified as lithology type: extrusive volcanic rock. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: extrusive volcanic rock. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial outwash and glacial lake sediment, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial outwash and glacial lake sediment, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial lake sediment, fine-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial lake sediment, fine-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, clayey. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, clayey. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, loamy. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, loamy. | 0% | 100 |
| Percentage of catchment area classified as lithology type: hydric, peat and muck. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: hydric, peat and muck. | 0% | 100 |
| Percentage of catchment area classified as lithology type: non-carbonate residual material. | 100% | 100 |
| Percentage of watershed area classified as as lithology type: non-carbonate residual material. | 57.4199853170687% | 100 |
| Percentage of catchment area classified as lithology type: saline like sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: saline like sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: silicic residual material. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: silicic residual material. | 42.5800146829313% | 100 |
| Percentage of catchment area classified as lithology type: water. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: water | 0% | 100 |

#### Landscape Layer: Mine Density Active Mines and Mineral Plants in the US
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced mines and mineral plants within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the riparian mask of the upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: National Anthropenic Barrier Dataset
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced dams within the local catchment (dams/square km). | 0dams/km² | 100 |
| Density of georeferenced dams within the total upstream watershed (dams/square km). | 0dams/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the local catchment (cubic meters/square km). | 0m³/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the total upstream watershed (cubic meters/square km). | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the local catchment (cubic meters/square km). | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the total upstream watershed (cubic meters/square km). | 0m³/km² | 100 |

#### Landscape Layer: National Atmospheric Deposition Program National Trends Network - Nitrogen Deposition
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Annual average of precipitation-weighted mean wet deposition for inorganic nitrogen concentration from nitrate and ammonium for year, 2008, within the local catchment. | .401904333791385kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean wet deposition for inorganic nitrogen concentration from nitrate and ammonium for year, 2008, within the upstream watershed. | .590328488878409m³/km² | 100 |
| Annual average of precipitation-weighted mean deposition for ammonium ion concentration for year, 2008, within the local catchment. | .227756479348523kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for ammonium ion concentration for year, 2008, within the total upstream watershed. | .330185657649953kg/ha/yr | 100 |
| Annual average of preciptiation-weighted mean deposition for nitrate ion concentration for year, 2008, within the local catchment. | .983374490437878kg/ha/yr | 100 |
| Annual average of preciptiation-weighted mean deposition for nitrate ion concentration for year, 2008, within the total upstream watershed. | 1.45850358788123kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for average sulfur and nitrogen concentration for year, 2008, within the local catchment. | 79.5310529718761kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for average sulfur and nitrogen concentration for year, 2008, within the total upstream watershed. | 119.136567723805kg/ha/yr | 100 |

#### Landscape Layer: 2006 National Land Cover Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of the local catchment area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 0% | 100 |
| Percentage of the upstream watershed area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 13.152405910123% | 100 |
| Percentage of the local catchment classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 2.89896320390323% | 100 |
| Percentage of the local catchment area classified as evergreen forest land cover. | 84.7988608045568% | 100 |
| Percentage of the upstream watershed area classified as evergreen forest land cover. | 71.2559740569134% | 100 |
| Percentage of the local catchment classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 95.4110898661568% | 100 |
| Percentage of the upstream watershed classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 87.3063630819272% | 100 |
| Percentage of the local catchment area classified as row crop land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as row crop land use. | 0% | 100 |
| Percentage of the local catchment classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as deciduous forest land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as deciduous forest land cover. | .0714698975598135% | 100 |
| Percentage of the local catchment classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | .178898150030494% | 100 |
| Percentage of the local catchment area classified as grassland/herbaceous land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as grassland/herbaceous land cover. | 3.77818077508375% | 100 |
| Percentage of the local catchment classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | .992071559260012% | 100 |
| Percentage of the local catchment area classified as pasture/hay land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as pasture/hay land use. | .0165304524968276% | 100 |
| Percentage of the local catchment classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as herbaceous wetland land cover. | .0024309488965923% | 100 |
| Percentage of the local catchment classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as ice/snow land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as ice/snow land cover. | 3.13884121527997% | 100 |
| Percentage of the local catchment classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 1.58162228095141% | 100 |
| Percentage of the local catchment area classified as mixed deciduous/evergreen forest land cover. | 2.31399074403702% | 100 |
| Percentage of the upstream watershed area classified as mixed deciduous/evergreen forest land cover. | .312620028101769% | 100 |
| Percentage of the local catchment classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 2.86806883365201% | 100 |
| Percentage of the upstream watershed classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | .630209392152877% | 100 |
| Percentage of the local catchment area classified as open water land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | .0252818685245599% | 100 |
| Percentage of the local catchment classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | .0243952022768855% | 100 |
| Percentage of the local catchment area classified as shrub/scrub land cover. | 10.5375578497686% | 100 |
| Percentage of the upstream watershed area classified as shrub/scrub land cover. | 7.80480452739922% | 100 |
| Percentage of the local catchment classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 5.6556210611913% | 100 |
| Percentage of the local catchment area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, low intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, low intensity land use. | .017989021834783% | 100 |
| Percentage of the local catchment classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | .0162634681845904% | 100 |
| Percentage of the local catchment area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, open space land use. | 2.31399074403702% | 100 |
| Percentage of the upstream watershed area classified as developed, open space land use. | .392841341689315% | 100 |
| Percentage of the local catchment classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | 1.52963671128107% | 100 |
| Percentage of the upstream watershed classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | .459442976214678% | 100 |
| Percentage of the local catchment area classified as woody wetland land cover. | .0355998576005696% | 100 |
| Percentage of the upstream watershed area classified as woody wetland land cover. | .0306299560970629% | 100 |
| Percentage of the local catchment classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .191204588910134% | 100 |
| Percentage of the upstream watershed classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .256149623907298% | 100 |

#### Landscape Layer: 2011 National Land Cover Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of the local catchment area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 0% | 100 |
| Percentage of the upstream watershed area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 13.1144831073361% | 100 |
| Percentage of the local catchment classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 2.81764586298028% | 100 |
| Percentage of the local catchment area classified as evergreen forest land cover. | 84.8344606621574% | 100 |
| Percentage of the upstream watershed area classified as evergreen forest land cover. | 71.2768802174241% | 100 |
| Percentage of the local catchment classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 95.0286806883365% | 100 |
| Percentage of the upstream watershed classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 86.8143931693434% | 100 |
| Percentage of the local catchment area classified as row crop land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as row crop land use. | 0% | 100 |
| Percentage of the local catchment classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as deciduous forest land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as deciduous forest land cover. | .0870279704980042% | 100 |
| Percentage of the local catchment classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | .309005895507217% | 100 |
| Percentage of the local catchment area classified as grassland/herbaceous land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as grassland/herbaceous land cover. | 3.77234649773192% | 100 |
| Percentage of the local catchment classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | .939215287660094% | 100 |
| Percentage of the local catchment area classified as pasture/hay land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as pasture/hay land use. | .0165304524968276% | 100 |
| Percentage of the local catchment classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the local catchment classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as ice/snow land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as ice/snow land cover. | 3.13884121527997% | 100 |
| Percentage of the local catchment classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 1.58162228095141% | 100 |
| Percentage of the local catchment area classified as mixed deciduous/evergreen forest land cover. | 2.31399074403702% | 100 |
| Percentage of the upstream watershed area classified as mixed deciduous/evergreen forest land cover. | .31553716677768% | 100 |
| Percentage of the local catchment classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 2.86806883365201% | 100 |
| Percent of the upstream watershed classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | .654604594429762% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | .0252818685245599% | 100 |
| Percentage of the local catchment classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | .0243952022768855% | 100 |
| Percentage of the local catchment area classified as shrub/scrub land cover. | 10.4307582769669% | 100 |
| Percentage of the upstream watershed area classified as shrub/scrub land cover. | 7.72117988535645% | 100 |
| Percentage of the local catchment classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershedclassified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 5.72474080097581% | 100 |
| Percentage of the local catchment area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, low intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, low intensity land use. | .017989021834783% | 100 |
| Percentage of the local catchment classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | .0162634681845904% | 100 |
| Percentage of the local catchment area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, open space land use. | 2.31399074403702% | 100 |
| Percentage of the upstream watershed area classified as developed, open space land use. | .392841341689315% | 100 |
| Percentage of the local catchment classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | 1.52963671128107% | 100 |
| Percentage of the upstream watershed classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | .459442976214678% | 100 |
| Percentage of the local catchment area classified as woody wetland land cover. | .106799572801709% | 100 |
| Percentage of the upstream watershed area classified as woody wetland land cover. | .121061255050296% | 100 |
| Percentage of the local catchment classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .573613766730401% | 100 |
| Percent of the upstream watershed classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .65867046147591% | 100 |

#### Landscape Layer: Nonnative LANDFIRE Vegetation
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the local catchment. | 2.66998932004272% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the total upstream watershed. | .713726596039498% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the riparian mask of the local catchment. | 1.52963671128107% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the riparian mask of the total upstream watershed. | .638341126245172% | 100 |

#### Landscape Layer: Pesticide
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Total pesticides per square kilometer within the local catchment (kilograms/square kilometer). | 0kg/km² | 100 |
| Total pesticides per square kilometer within the total upstream watershed (kilograms/square kilometer). | 0kg/km² | 100 |

#### Landscape Layer: PRISM Data 2008-09
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean 2008 precipitation (mm) within the local catchment. | 718.8232489mm | 100 |
| Mean 2008 precipitation (mm) within the total upstream watershed. | 1198.04654526mm | 100 |
| Mean 2009 precipitation (mm) within the local catchment. | 932.588003179mm | 100 |
| Mean 2009 precipitation (mm) within the total upstream watershed. | 1455.15271215mm | 100 |
| Mean 2008 air temperature (Celcius) within the local catchment. | 8.4908489738°C | 100 |
| Mean 2008 air temperature (Celcius) within the total upstream watershed. | 5.0326654059°C | 100 |
| Mean 2009 air temperature (Celcius) within the local catchment. | 9.29403854852°C | 100 |
| Mean 2009 air temperature (Celcius) within the total upstream watershed. | 5.91343056194°C | 100 |

#### Landscape Layer: PRISM Normals Data
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| 30-year average annual normal precipitation (mm) within the local catchment. | 1063.95128362mm | 100 |
| 30-year average annual normal precipitation (mm) within the upstream watershed. | 1558.58192373mm | 100 |
| 30-year average annual normal maximum air temperature (Celcius) within the local catchment. | 13.5465186446°C | 100 |
| 30-year average annual normal maximum air temperature (Celcius) within the upstream watershed. | 9.50995546973°C | 100 |
| 30-year average annual normal mean air temperature (Celcius) within the local catchment. | 9.1920788586°C | 100 |
| 30-year average annual normal mean air temperature (Celcius) within the upstream watershed. | 5.29888636597°C | 100 |
| 30-year average annual normal minimum air temperature (Celcius) within the local catchment. | 4.83465089199°C | 100 |
| 30-year average annual normal minimum air temperature (Celcius) within the upstream watershed. | 1.08292704734°C | 100 |

#### Landscape Layer: Reference Stream Temperature Predictions
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Predicted annual stream temperature (Celcius) for year 2008. | 8.0742748145°C |  |
| Predicted annual stream temperature (Celcius) for year 2009. | 8.03306204508°C |  |
| Predicted annual stream temperature (Celcius) for year 2013. | 8.13708438617°C |  |
| Predicted annual stream temperature (Celcius) for year 2014. | 8.82835227728°C |  |
| Predicted summer stream temperature (Celcius) for year 2008. | 13.365178241°C |  |
| Predicted summer stream temperature (Celcius) for year 2009. | 13.9685128657°C |  |
| Predicted summer stream temperature (Celcius) for year 2013. | 13.9022800001°C |  |
| Predicted summer stream temperature (Celcius) for year 2014. | 14.2323948091°C |  |
| Predicted winter stream temperature (Celcius) for year 2008. | 4.121683584°C |  |
| Predicted winter stream temperature (Celcius) for year 2009. | 4.35265688113°C |  |
| Predicted winter stream temperature (Celcius) for year 2013. | 4.33874321091°C |  |
| Predicted winter stream temperature (Celcius) for year 2014. | 4.21071690164°C |  |

#### Landscape Layer: 2010 US Census Road Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Average density of roads per square kilometer within the local catchment (kilometer of road/square kilometer). | 1.25095151482022km/km² | 100 |
| Average density of roads per square kilometer within the total upstream watershed (kilometer of road/square kilometer). | .12258975399154km/km² | 100 |
| Average density of roads per square kilometer within the riparian mask of the local catchment (kilometer of road/square kilometer). | .851576749499044km/km² | 100 |
| Average density of roads per square kilometer within the riparian mask of the total upstream watershed (kilometer of road/square kilometer). | .114404952418784km/km² | 100 |

#### Landscape Layer: Road and Stream Intersections
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of rdstrcrs values (crossings / square kilometer) within the local catchment. | .39555397334crossings/km² | 100 |
| Mean of rdstrcrs values (crossings / square kilometer) within the total upstream watershed. | .0108042173182crossings/km² | 100 |
| Rdstrcrs values multiplied by NHD slope value from elevslope.dbf (crossings * slope / square kilometer) within the local catchment. | .00969542739607crossings * slope/km² | 100 |
| Rdstrcrs values multiplied by NHD slope value from elevslope.dbf (crossings * slope / square kilometer) within the total upstream watershed. | .000414629126333crossings * slope/km² | 100 |

#### Landscape Layer: Runoff
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all runoff values within the local catchment. | 1575mm | 100 |
| Mean of all runoff values within the upstream watershed. | 1585.47141447193mm | 100 |

#### Landscape Layer: State Soil Geographic Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all clay values within the local catchment. | 14.55% | 100 |
| Mean of all clay values within the upstream watershed. | 14.7501251938682% | 100 |
| Mean of all sand values within the local catchment. | 37.9500071199715% | 100 |
| Mean of all sand values within the upstream watershed. | 50.5249560727534% | 100 |
| Mean of all organic matter values within the local catchment. | .73% | 100 |
| Mean of all organic matter values within the upstream watershed. | 1.66460171231162% | 77.6853476986 |
| Mean of all permeability of soils values within the local catchment. | 3.3cm/hour | 100 |
| Mean of all permeability of soils values within the upstream watershed. | 4.2198934206162cm/hour | 77.6853476986 |
| Mean of all depth to bedrock of soils values within the local catchment. | 139.7cm | 100 |
| Mean of all depth to bedrock of soils values within the upstream watershed. | 88.0068741512138cm | 77.6853476986 |
| Mean of all seasonal water table depth of soils values within the local catchment. | 179.37cm | 100 |
| Mean of all seasonal water table depth of soils values within the upstream watershed. | 182.227368058728cm | 77.6853476986 |

#### Landscape Layer: 2010 US Census Housing Unit and Population Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all housing units per square kilometer values within the local catchment. | 3.39011838957992housing units/km² | 100 |
| Mean of all housing units per square kilometer values within the upstream watershed. | 2.08106563179876housing units/km² | 100 |
| Mean of all housing units per square kilometer within the local catchment within a 100-m wide buffer of the NHD stream lines. | 2.78983565783939housing units/km² | 100 |
| Mean of all housing units per square kilometer within the upstream watershed within a 100-m wide buffer of the NHD stream lines. | 2.2043870312096housing units/km² | 100 |
| Mean of all 2010 population per square kilometer values within the local catchment. | 6.565384899466people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the upstream watershed. | 3.51321833257326people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the local catchment within a 100-m wide buffer of the NHD stream lines. | 5.70105569300191people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the upstream watershed within a 100-m wide buffer of the NHD stream lines. | 3.89144162234194people/km² | 100 |

#### Landscape Layer: Wetness Index
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean Composite Topographic Index (CTI) [Wetness Index] within the local catchment. | 295.22463510146 | 100 |
| Mean Composite Topographic Index (CTI) [Wetness Index] within the upstream watershed. | 245.500503206422 | 100 |



[DEBUG] Hover event: mouseover, feature: {'id': '0', 'type': 'Feature', 'properties': {'type': 'hydrolocation', 'source': 'nwissite', 'sourceName': 'NWIS Surface Water Sites', 'identifier': 'USGS-12047720', 'name': 'DUNGENESS RIVER ABV GRAY WOLF RIVER NEAR BLYN, WA', 'uri': 'https://waterdata.usgs.gov/monitoring-location/12047720', 'comid': '23997454', 'reachcode': '17110020000196', 'measure': '11.1022000000', 'navigation': 'https://api.water.usgs.gov/nldi/linked-data/nwissite/USGS-12047720/navigation'}, 'geometry': {'type': 'Point', 'coordinates': [-123.1098957331627, 47.97425807535966]}}, id: 0, coords: None
[DEBUG] Hover event: mouseover, feature: {'id': '1', 'type': 'Feature', 'properties': {'type': 'hydrolocation', 'source': 'nwissite', 'sourceName': 'NWIS Surface Water Sites', 'identifier': 'USGS-12047780', 'name': 'DUNGENESS RIVER BLW GRAY WOLF RIVER NEAR BLYN, WA', 'uri': 'https://waterdata.usgs.gov/monitoring-location/12047780', 'comid': '23997390', 'reachcode': '1711002000016